In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
from sklearn.pipeline import Pipeline
import _tensorflow as tf
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.model import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

In [ ]:
df=pd.read_csv('Churn_Modelling.csv')

In [ ]:
#preprocess the ireleveant data 
df=df.drop(['RowNumber','CustomerId','Surname'],axis=1) 
#change categorical do encoding 
le_gender=LabelEncoder() 
df['Gender']=le_gender.fit_transform(df['Gender'])
 #use one hot encoding 
from sklearn.preprocessing import OneHotEncoder 
on_geo=OneHotEncoder() 
geo_en=on_geo.fit_transform(df[['Geography']]) 
on_geo.get_feature_names_out(['Geography']) 
geo_en.toarray() 
geo_encoded_df=pd.DataFrame(geo_en.toarray(),columns=on_geo.get_feature_names_out(['Geography']))

In [ ]:
import pickle
df=pd.concat([df.drop(['Geography'],axis=1),geo_encoded_df],axis=1)
 #save the encoders and scalers 
with open('LabelEncoder_gender.pkl','wb') as file:
    pickle.dump(le_gender,file) 
with open('OneHotEncoder_geography1.pkl','wb') as file:
    pickle.dump(on_geo,file) 
#divide the data into independent and dependent variables 
x=df.drop('Exited',axis=1) 
y=df['Exited'] 
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42) 
scaler=StandardScaler() 
x_train=scaler.fit_transform(x_train) 
x_test=scaler.transform(x_test) 
with open('scaler.pkl','wb')as file: 
    pickle.dump(scaler,file)

In [ ]:
#define a function to create a model and try different parameter(keras classifer)
def create_model(neurons=32,layers=1):
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_shape=(x_train.shape[1],)))
    for _ in range(layers - 1):
        model.add(Dense(neurons, activation='relu'))

    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer="Adam", loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
#create a keras classifier
model=KerasClassifier(build_fn=create_model,epochs=50,batch_size=10,verbose=0)


In [ ]:
#define a grid search parameters
param_grid={
    'neurons':[16,32,64,128],
    'layers':[1,2],
    'epochs':[50,100]
}

In [ ]:
#perform grid search
grid=GridSearchCV(estimator=model,param_grid=param_grid,n_jobs=-1,cv=3)
grid_result=grid.fit(x_train,y_train)

#print the best parameters
print("best: %f using %s" %(grid_result.best_score_,grid_result.best_params_))